In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install pyspark==3.2.0 kafka-python==2.0.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 22.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805917 sha256=6dfe8aca0c0c9ac796df7cab6e30817ecfd8e44f24962d82b7ba7397df3b8af6
  Stored in directory: /root/.cache/pip/wheels/32/97/d3/8b6d964c8700e4fbb561c71638a92ec55dac9be51eb5fea86d
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.4.0
    Uninstalling pyspark-3.4.0:
      Successfully uninstalled pyspark-3.4.0


In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col,count,when,isnan
from pyspark.ml.feature import PCA
from pyspark.ml.classification import LinearSVC
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt
from pyspark.ml.evaluation import ClusteringEvaluator
import pickle
import matplotlib.pyplot as plt

In [ ]:
spark=SparkSession.builder.appName('my_app').getOrCreate()
df=spark.read.csv('/content/uber-raw-data-aug14.csv',header=True,inferSchema=True)

In [ ]:
#!pip install kafka-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
df.show()

+----------------+-------+--------+------+
|       Date/Time|    Lat|     Lon|  Base|
+----------------+-------+--------+------+
|8/1/2014 0:03:00|40.7366|-73.9906|B02512|
|8/1/2014 0:09:00| 40.726|-73.9918|B02512|
|8/1/2014 0:12:00|40.7209|-74.0507|B02512|
|8/1/2014 0:12:00|40.7387|-73.9856|B02512|
|8/1/2014 0:12:00|40.7323|-74.0077|B02512|
|8/1/2014 0:13:00|40.7349|-74.0033|B02512|
|8/1/2014 0:15:00|40.7279|-73.9542|B02512|
|8/1/2014 0:17:00| 40.721|-73.9937|B02512|
|8/1/2014 0:19:00|40.7195| -74.006|B02512|
|8/1/2014 0:20:00|40.7448|-73.9799|B02512|
|8/1/2014 0:21:00|40.7399|-74.0057|B02512|
|8/1/2014 0:25:00|40.7651|-73.9683|B02512|
|8/1/2014 0:27:00|40.7354|-74.0081|B02512|
|8/1/2014 0:29:00|40.7339|-74.0028|B02512|
|8/1/2014 0:29:00|40.7364|-74.0301|B02512|
|8/1/2014 0:29:00|40.7364|-74.0301|B02512|
|8/1/2014 0:30:00|40.7252|-73.9516|B02512|
|8/1/2014 0:30:00|40.7433| -73.986|B02512|
|8/1/2014 0:34:00|40.7437|-73.9884|B02512|
|8/1/2014 0:36:00|40.7406|-74.0077|B02512|
+----------

In [ ]:
nr= df.count()
nc = len(df.columns)
print("Number of rows: ", nr)
print("Number of columns: ", nc)

Number of rows:  829275
Number of columns:  4


In [ ]:
df.describe().show()

+-------+----------------+--------------------+-------------------+------+
|summary|       Date/Time|                 Lat|                Lon|  Base|
+-------+----------------+--------------------+-------------------+------+
|  count|          829275|              829275|             829275|829275|
|   mean|            null|   40.73778073582462| -73.97016031317625|  null|
| stddev|            null|0.043628060846868946|0.06148272834518351|  null|
|    min|8/1/2014 0:00:00|             39.6569|           -74.7737|B02512|
|    max|8/9/2014 9:59:00|             41.3182|           -72.3359|B02764|
+-------+----------------+--------------------+-------------------+------+



In [ ]:
null = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
null.show()

+---------+---+---+----+
|Date/Time|Lat|Lon|Base|
+---------+---+---+----+
|        0|  0|  0|   0|
+---------+---+---+----+



In [ ]:
string_cols = [c for c, t in df.dtypes if t == "string"]
indexers = {c: StringIndexer(inputCol=c, outputCol=c+"_index") for c in string_cols}
indexed_df = df
for c, indexer in indexers.items():
    indexed_df = indexer.fit(indexed_df).transform(indexed_df)
numeric_cols = [c for c, t in indexed_df.dtypes if t in ["int", "double", "float", "long"]]
numeric_df = indexed_df.select(numeric_cols)
numeric_df.show()

+-------+--------+---------------+----------+
|    Lat|     Lon|Date/Time_index|Base_index|
+-------+--------+---------------+----------+
|40.7366|-73.9906|        20173.0|       4.0|
| 40.726|-73.9918|        34403.0|       4.0|
|40.7209|-74.0507|        25098.0|       4.0|
|40.7387|-73.9856|        25098.0|       4.0|
|40.7323|-74.0077|        25098.0|       4.0|
|40.7349|-74.0033|        29513.0|       4.0|
|40.7279|-73.9542|        20174.0|       4.0|
| 40.721|-73.9937|        29515.0|       4.0|
|40.7195| -74.006|        26643.0|       4.0|
|40.7448|-73.9799|        29516.0|       4.0|
|40.7399|-74.0057|        32031.0|       4.0|
|40.7651|-73.9683|        29518.0|       4.0|
|40.7354|-74.0081|        30819.0|       4.0|
|40.7339|-74.0028|        20175.0|       4.0|
|40.7364|-74.0301|        20175.0|       4.0|
|40.7364|-74.0301|        20175.0|       4.0|
|40.7252|-73.9516|        33228.0|       4.0|
|40.7433| -73.986|        33228.0|       4.0|
|40.7437|-73.9884|        26644.0|

In [ ]:
assembler = VectorAssembler(inputCols=['Lat', 'Lon'], outputCol='features')
data = assembler.transform(df.select('Lat', 'Lon'))

In [ ]:
(trainingData, testData) = data.randomSplit([0.8, 0.2], seed=0)

In [ ]:
testdatatocsv = testData.select('Lat','Lon')
testdatatocsv.write.csv('test.csv', header=True, mode='overwrite')

In [ ]:
#!pip install kafka-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#import json
#from pyspark.sql.functions import to_json, struct
#from time import sleep
#from kafka import KafkaProducer

#producer = KafkaProducer(bootstrap_servers=['localhost:9092'], 
#                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

#df_json = testData.select(to_json(struct([testData[x] for x in testData.columns])).alias("value")).collect()

#for data in df_json:
#    producer.send('uber_data', value=data['value'])
#    sleep(1)
#    print(f"{data['value']}")


In [ ]:
cost = []
for i in range(2, 11):
    kmeans = KMeans(k=i, seed=100)
    model = kmeans.fit(data.select('Lat', 'Lon' , 'features'))
    predictions = model.transform(data)
    evaluator = ClusteringEvaluator()
    silhouette = evaluator.evaluate(predictions)
    print(f'Cost {i} : ' + str(silhouette))
    cost.append(silhouette)  
#model.write().overwrite().save("kmeans_model")    
    

In [ ]:
plt.plot(range(2, 11), cost)
plt.show()

In [ ]:
assembler = VectorAssembler(inputCols=["Lat", "Lon"], outputCol="features")
trainingData = assembler.transform(trainingData).select("features")
kmeans = KMeans().setK(5).setSeed(20)
model = kmeans.fit(trainingData)
model.save("kmeans5.model")

In [ ]:
kmeans = KMeans().setK(5).setSeed(20)
model = kmeans.fit(trainingData)
model.save("kmeans5.model")

In [ ]:
#from confluent_kafka import Producer, KafkaError

# تنظیمات کانفیگ کافکا
#conf = {
  #  'bootstrap.servers': 'localhost:9092', # آدرس سرورهای بوت استرپ
 #   'client.id': 'python-producer'
#}

# ایجاد یک تولیدکننده کافکا
#producer = Producer(conf)
#file_path='/content/output.csv/part-00000-be01dc14-085d-4a48-9cc5-75caf43aa79b-c000.csv'
# باز کردن فایل CSV و خواندن داده‌ها
#with open(file_path, 'r') as f:
 #   for line in f:
 #       # فرستادن هر خط به کافکا به عنوان یک پیام
  #      producer.produce('my_topic', line.encode('utf-8'))
#
# فلش کردن پیام‌ها در کافکا
#producer.flush()

In [ ]:
#from pyspark.streaming import StreamingContext
#from pyspark.streaming.kafka import KafkaUtils
# تعریف تابع پردازش برای استریم
#def process_stream(stream):
    # پردازش داده های رسیده به استریم
#    stream.pprint()

# تنظیمات اسپارک
#appName = "KafkaSparkStream"
#master = "local[*]"
#batchDuration = 5

# تنظیمات کافکا
#kafkaParams = {
  #  "metadata.broker.list": "localhost:9092",
 #   "auto.offset.reset": "smallest"
#}
#topic = "my_topic"

# ایجاد شیء StreamingContext
#ssc = StreamingContext(master, batchDuration)

# خواندن داده های کافکا به عنوان یک DStream
#kafkaStream = KafkaUtils.createDirectStream(ssc, [topic], kafkaParams)

# پردازش داده های دریافت شده از کافکا
#kafkaStream.foreachRDD(lambda rdd: process_stream(rdd))

# شروع استریم
#ssc.start()
#ssc.awaitTermination()


In [ ]:
model.save('kmeans')

In [ ]:
predictions.show()

+-------+--------+------------------+----------+
|    Lat|     Lon|          features|prediction|
+-------+--------+------------------+----------+
|40.7366|-73.9906|[40.7366,-73.9906]|         4|
| 40.726|-73.9918| [40.726,-73.9918]|         4|
|40.7209|-74.0507|[40.7209,-74.0507]|         4|
|40.7387|-73.9856|[40.7387,-73.9856]|         0|
|40.7323|-74.0077|[40.7323,-74.0077]|         4|
|40.7349|-74.0033|[40.7349,-74.0033]|         4|
|40.7279|-73.9542|[40.7279,-73.9542]|         0|
| 40.721|-73.9937| [40.721,-73.9937]|         4|
|40.7195| -74.006| [40.7195,-74.006]|         4|
|40.7448|-73.9799|[40.7448,-73.9799]|         0|
|40.7399|-74.0057|[40.7399,-74.0057]|         4|
|40.7651|-73.9683|[40.7651,-73.9683]|         0|
|40.7354|-74.0081|[40.7354,-74.0081]|         4|
|40.7339|-74.0028|[40.7339,-74.0028]|         4|
|40.7364|-74.0301|[40.7364,-74.0301]|         4|
|40.7364|-74.0301|[40.7364,-74.0301]|         4|
|40.7252|-73.9516|[40.7252,-73.9516]|         8|
|40.7433| -73.986| [

In [ ]:
#import pandas as pd
#print(df.dtypes)

[('Date/Time', 'string'), ('Lat', 'double'), ('Lon', 'double'), ('Base', 'string')]


In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from pyspark.sql import SparkSession
#from pyspark.sql.functions import *

#spark = SparkSession.builder.appName("csv_to_kafka").getOrCreate()

# خواندن داده های CSV
#df = spark.read.format("uber_data.csv").option("header", "true").load("output.csv")

# تبدیل داده ها به رشته JSON
#df_json = df.select(to_json(struct("*")).alias("value"))

# ارسال داده ها به Kafka
#df_json.write.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("topic", "test_topic").save()

# بستن SparkSession
#spark.stop()

In [ ]:
!pwd
!ls -r

/content
uber-raw-data-aug14.csv  sample_data  output.csv
